# Reddit stock market prediction

### Import libraries

In [2]:
# pull libraries
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split

### Preprocess data

In [3]:
# pull data
djia_data = pd.read_csv('./stocknews/Combined_News_DJIA.csv', parse_dates=True)

djia_data['combined'] = djia_data[djia_data.columns[2:]].apply(
    lambda x: ' '.join(x.astype(str)),
    axis=1
)

def sanitise_row(row):
    return re.sub('[^A-Za-z ]+', '', row.replace("b\"", "").replace("b'", ""))

def remove_stop_words(sentence):
    sentence = sanitise_row(sentence)
    stop_words = set(stopwords.words('english')) 
    word_tokens = word_tokenize(sentence) 
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    return ' '.join(filtered_sentence)

djia_data['combined'] = djia_data['combined'].map(lambda x: remove_stop_words(x))

# print(type(djia_data['combined']))

In [4]:
# Ratio of label 1s and 0s
data_label = np.array(djia_data['Label'])

print(np.sum(data_label == 1) / data_label.shape[0])

0.5354449472096531


### Preprocess data

In [5]:
# prepare train and test data
all_data = djia_data['combined']
data_label = np.array(djia_data['Label'])

# vectorizer = CountVectorizer()
# X = vectorizer.fit_transform(all_data)
# print(vectorizer.get_feature_names())
# print(X.shape)

X_train, X_test, y_train, y_test = train_test_split(
    all_data, data_label, test_size=0.50, random_state=23)
# we also need dictionary that maps word to number of occurences -> 
# to handle cases where word is not found in the training corpus

In [6]:
# We need to convert training data into a vector for LSTM
# how does the vector look like?
# need to build word_index first!

def gen_word_indexes(data): 
    word_index = {}
    word_index["<PAD>"] = 0
    word_index["<START>"] = 1
    word_index["<UNK>"] = 2  # unknown
    word_index["<UNUSED>"] = 3

    index = 4
    # loop through everything in all_data
    for row in data:
        for word in row.split():
            word = word.lower()
            if word not in word_index:
                word_index[word] = index
                index += 1

    reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
    return (word_index, reverse_word_index)


def vectorise_text_data(data, word_index):
    vectorised_data = []
    for row in data:
        current_row = []
        for word in row.split():
            word = word.lower()
            if word not in word_index:
                current_row.append(word_index["<UNUSED>"])
            else:
                current_row.append(word_index[word])
        vectorised_data.append(current_row)
    return vectorised_data

# really slow :(
def aggregate_previous_days(num_days, data):
    for i in range(num_days, len(data) - num_days):
        for d in range(i, i + num_days):
            data[i] += ' ' + data[d]
    return data

In [65]:
(word_index, reverse_word_index) = gen_word_indexes(all_data)
# save word_index somewhere
import pickle
pickle_out = open("word_index.pickle","wb")
pickle.dump(word_index, pickle_out)
pickle_out.close()

X_vectorised = vectorise_text_data(all_data, word_index)

In [58]:
from keras.preprocessing import sequence

# number of most-frequent words to use
nb_words = 40000
# cut texts after this number of words
maxlen = 600

X_v_train, X_v_test, y_v_train, y_v_test = train_test_split(
    X_vectorised, data_label, test_size=0.50, random_state=23)

X_v_pad_train = sequence.pad_sequences(X_v_train,
                                       value=word_index["<PAD>"],
                                       padding='post',
                                       maxlen=maxlen)
X_v_pad_test = sequence.pad_sequences(X_v_test, 
                                      value=word_index["<PAD>"],
                                      padding='post',
                                      maxlen=maxlen)

print(X_v_pad_train.shape, X_v_pad_test.shape)
print(X_v_pad_train[6])

(994, 600) (995, 600)
[  444  2131  6102  3963  5816  1949  1096 10617  1638  2131  3239  2961
  3963  5816  1075   225   392  1949 10306  1802 25540  6158   100  2814
  7770 28459  4976   306   307  1623   340  2313  1646  2379  2218  1808
    89 25884 28460  1623   180 12995 12996  2982   875  5419  5420   953
  4494  9126 10574  3310  5419  5420  1219  7572  4494  2103  9126  1258
   692 11386 23245  1049  1802  3925   505 22284  7422  2015  1507  2149
  8131  4398  1518 20111   505   365  2971  8084  7901  1469  8583   284
  5073  8084  3488   125   625 12480  1029   231  2711   330  1623    93
  1095  8553   546   142  2922   365  2806  1522 28461 28462  4761  1052
 28463  2433  1098  3182 28464  1096 14717 25209 15720    89   184  1623
  2157    11  1179    12  1955    26 27080  1222  1761  5100   580  1829
  1142   758  1044  1519    58  1002   310  2295   374    54  1142   758
  7566 10179   712  8397  1870  3760  2207   431  2220   310  8749  1044
   444     6    66   634    5

Now what? We train!

### Training using Naive Bayes

In [18]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression

# LogisticRegression(random_state=0, solver='lbfgs')

v = CountVectorizer()

text_clf = Pipeline([
    ('vect', v),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB())
])

text_clf.fit(X_train, y_train)

predicted = text_clf.predict(X_test)
np.mean(predicted == y_test)  

/Users/muhammad/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


0.5386934673366834

### Training using LSTM

In [59]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, Dropout, SpatialDropout1D
from keras.layers import LSTM

EMBEDDING_DIM = 128 # dimension for dense embeddings for each token
LSTM_DIM = 64 # total LSTM units

model = Sequential()
model.add(Embedding(input_dim=nb_words, output_dim=EMBEDDING_DIM, input_length=maxlen))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(LSTM_DIM, dropout=0.8, recurrent_dropout=0.8))
model.add(Dense(1, activation="sigmoid"))

model.compile(loss="binary_crossentropy", optimizer="adam",
              metrics=["accuracy"])

In [61]:
batch_size = 64
model.fit(X_v_pad_train, y_v_train, epochs=7, batch_size=batch_size, 
          shuffle=True, validation_split=0.3, verbose=1)

Train on 695 samples, validate on 299 samples
Epoch 1/7
695/695 [==============================] - 13s 18ms/step - loss: 0.6980 - acc: 0.4935 - val_loss: 0.6908 - val_acc: 0.5585
Epoch 2/7
695/695 [==============================] - 13s 18ms/step - loss: 0.6995 - acc: 0.4647 - val_loss: 0.6915 - val_acc: 0.5585
Epoch 3/7
695/695 [==============================] - 13s 18ms/step - loss: 0.6966 - acc: 0.5007 - val_loss: 0.6914 - val_acc: 0.5585
Epoch 4/7
695/695 [==============================] - 13s 19ms/step - loss: 0.6994 - acc: 0.4964 - val_loss: 0.6912 - val_acc: 0.5585
Epoch 5/7
695/695 [==============================] - 13s 18ms/step - loss: 0.6914 - acc: 0.5295 - val_loss: 0.6915 - val_acc: 0.5585
Epoch 6/7
695/695 [==============================] - 13s 18ms/step - loss: 0.6959 - acc: 0.4964 - val_loss: 0.6918 - val_acc: 0.5585
Epoch 7/7
695/695 [==============================] - 14s 21ms/step - loss: 0.6961 - acc: 0.4777 - val_loss: 0.6919 - val_acc: 0.5585


In [63]:
scores = model.evaluate(X_v_pad_test, y_v_test, verbose=2)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

acc: 55.08%


In [52]:
# Save the model
from keras.models import model_from_json

# serialize model to JSON
model_json = model.to_json()
with open("model2.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model2.h5")
print("Saved model to disk")

Saved model to disk
